# Simulation & Visualisation Examples 

### Visualisation of a Protein

https://www.rcsb.org/

In [20]:
import nglview
view = nglview.show_pdbid("3pqr")  # load "3pqr" from RCSB PDB
view

NGLWidget()

In [66]:
def show_system(atoms):
    v = view(atoms, viewer="ngl")
    v.view._remote_call("setSize", target="Widget", args=["700px","700px"])
    v.view.add_ball_and_stick()
    v.view.background="black"
    return v

###  Molecular Dynamics Simulation of Acetonitrile

In [ ]:
%%time
import ase.units as units
from ase.calculators.acn import ACN
from ase.constraints import FixLinearTriatomic
from ase.io import Trajectory,write
from ase.md import Langevin

atoms = read("acn.xyz")
atoms.constraints = FixLinearTriatomic(
    triples=[(3 * i, 3 * i + 1, 3 * i + 2)
             for i in range(nm)])
tag = 'acn'
atoms.calc = ACN(rc=np.min(np.diag(atoms.cell)) / 2)

# Create Langevin object
md = Langevin(atoms, 1 * units.fs,
              temperature=300 * units.kB,
              friction=0.01,
              logfile=tag + '.log')

traj = Trajectory(tag + '.traj', 'w', atoms)
md.attach(traj.write, interval=10)
md.run(20000)

In [ ]:
traj = Trajectory("acn.traj")
show_system(traj)